# Exploring eBay Car Sales Data

Whether it's a first-time buyer looking for their first used car or a classic car enthusiast tracking down that rare vintage model, used car sites simplify the shopping experience with detailed, searchable listings, car reviews, buyers guides, and more.  These sites don't just help buyers find cars; they are an excellent resource for car owners to find buyers too.

## Data Overview
We will work with a dataset of used cars from **eBay Kleinanzeigen**, a [classifieds](https://en.wikipedia.org/wiki/Classified_advertising) section of the German eBay website. The original dataset is not available on kaggle anymore, but you can find it [here](https://data.world/data-society/used-cars-data). 

Our version of the dataset has been intentionally *dirtied* (the original version was cleaned to make the data easier to work with), with a sample of 50,000 data points from the full dataset *(credits to [DataQuest](https://www.dataquest.io))*.

Here is an overview of the columns in the dataset:

- `dateCrawled`:- When this ad was first crawled. All field-values are taken from this date.

- `name`:- Name of the car.

- `seller`:- Whether the seller is private or a dealer.

- `offerType`:- The type of listing

- `price`:- The price on the ad to sell the car.

- `abtest`:- Whether the listing is included in an A/B test.

- `vehicleType`:- The vehicle Type.

- `yearOfRegistration` - The year in which the car was first registered.

- `gearbox`:- The transmission type.

- `powerPS`:- The power of the car in PS.

- `model`:- The car model name.

- `kilometer`:- How many kilometers the car has driven.

- `monthOfRegistration`:- The month in which the car was first registered.

- `fuelType`:- What type of fuel the car uses.

- `brand`:- The brand of the car.

- `notRepairedDamage`:- If the car has a damage which is not yet repaired.

- `dateCreated`:- The date on which the eBay listing was created.

- `nrOfPictures`:- The number of pictures in the ad.

- `postalCode`:- The postal code for the location of the vehicle.

- `lastSeenOnline`:- When the crawler saw this ad last online.

Our goal is to clean the data and analyze the included used car listings.

### Importing Useful Libraries

Let's start by importing the libraries we need. We will import the `Numpy` library to perform quick, vectorized operations on our data; the `Pandas` library to extend the functionalities of Numpy even further; then some `Plotly` libraries to help us build visualizations.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

### Basic Data Exploration

Next, we will read our dataset into a Pandas dataframe, then explore it further to identify where cleaning would be neccessary.

In [2]:
# Import dataset
autos = pd.read_csv('autos.csv', encoding='Latin-1') 

# Explore dataset
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


#### Observations

- *Our Dataset contains 20 columns and 50,000 rows. Most of the columns contain object or string data, a few columns contain integers.*

- *The `price` and `odometer` columns contain numeric values stored as text.* 

- *We have 5 columns with null values (`vehicleType`, `gearbox`, `model`, `fuelType` and `notRepairedDamage`), although their proportion is significantly low (less than 10% in their respective columns).*

- *On closer examination, we'd observe that columns such as `dateCrawled`, `dateCreated` and `lastSeen` are saved as object/string data instead of dates.*

- *6 columns have string data recorded in German language. These columns are - `seller`, `offerType`, `vehicleType`, `gear_box`, `fuelType` and `notRepairedDamage`.*

- *Rather than Python's preferred snakecase, the column names follow camelcase convention.*

Let's explore the column names in a bit more detail:

In [3]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

## Data Cleaning

### Cleaning Column Names

We will start by converting the column names from camelcase to snakecase. Next, we will rename some columns to make them more descriptive and easier to work with.

In [4]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
               'vehicle_type', 'registration_year', 'gear_box', 'powerPS', 'model',
               'odometer', 'registration_month', 'fuel_type', 'brand',
               'unrepaired_damage', 'ad_created', 'num_pictures', 'postal_code',
               'last_seen']
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gear_box,powerPS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Let's do some exploration to determine what other cleaning tasks need to be done. We will try to identify:

- Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis.

- Data that is not stored in the best format for analysis. They may need to be cleaned, converted or both.

We will use the `DataFrame.describe()` method (with the _include='all'_ argument to get information for both categorical and numeric columns). If any columns need us to take a closer look, we will use the `Series.value_counts()` and `Series.head()` to glean more insights.

In [5]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gear_box,powerPS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


#### Observations

*The columns `seller`, `offer_type`, `ab_test`, `gear_box`, and `unrepaired_damage` seem to have only 2 unique values. `num_pictures` contains 50,000 values, but has a mean of 0. That seems unusual!*

Let's deal with the columns having 2 unique values:

In [6]:
# loop through each column and compute the frequency of the unique values

for item in ['seller', 'offer_type', 'ab_test', 'gear_box', 'unrepaired_damage']:
    print('-'*30)
    print(autos[item].value_counts())
    print('-'*30)

------------------------------
privat        49999
gewerblich        1
Name: seller, dtype: int64
------------------------------
------------------------------
Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64
------------------------------
------------------------------
test       25756
control    24244
Name: ab_test, dtype: int64
------------------------------
------------------------------
manuell      36993
automatik    10327
Name: gear_box, dtype: int64
------------------------------
------------------------------
nein    35232
ja       4939
Name: unrepaired_damage, dtype: int64
------------------------------


#### Observations

*The Seller and offer_type columns are heavily one-sided:*

- *From the seller column/series, Privat means 'private' and gewerblick means 'commercial'. This tells us that most records in the dataset were from private sellers.*

- *From the offer_type column/series, Angebot stands for 'offer', while gesuch stands for 'request'. This tells us that the dataset majorly contains records from people who offered their car for sale.*

These columns are far too one-sided to add useful information to our analysis. We will drop these columns, but first, lets explore the `num_pictures` column:

In [7]:
autos['num_pictures'].value_counts()

0    50000
Name: num_pictures, dtype: int64

With all data for `num_pictures` being 0, It appears that none of the listings contain any pictures. We will drop this column along with `seller` and `offer_type`

In [8]:
print(autos.shape)
autos = autos.drop(['seller', 'offer_type', 'num_pictures'], axis=1)
print(autos.shape)

(50000, 20)
(50000, 17)


### Cleaning Non English Columns

In the previous observation, we had to manually interpret the meaning of the terms in the `seller` and `offer_type` columns. Repeating this process for all affected columns will be cumbersome in the long run. 

There are six columns recorded in German. However, we have already dropped two of the affected columns. We will now explore the other four in more detail:

In [9]:
german_columns = ['vehicle_type', 'gear_box','fuel_type', 'unrepaired_damage']

# print the unique german words in each column of interest
for column in german_columns:
    print('')
    print(autos[column].unique())


['bus' 'limousine' 'kleinwagen' 'kombi' nan 'coupe' 'suv' 'cabrio'
 'andere']

['manuell' 'automatik' nan]

['lpg' 'benzin' 'diesel' nan 'cng' 'hybrid' 'elektro' 'andere']

['nein' nan 'ja']


We can use this information to build a dictionary of translations for each German word. Then, we will define a function `convert_language()` that will iterate through each of the affected columns and translate their values using records from the dictionary. The `series.map()` method will handle the translation process for us:

In [10]:
# translator function
def translate_column(df,column,value_dict):
    '''translates values in df column using value_dict as reference'''
    df[column] = df[column].map(value_dict)


# A dictionary of translations
translations ={# vehicle type terms
               'bus':'bus',
               'limousine':'limousine',
               'kleinwagen': 'small car',
               'kombi': 'combo',
               'coupe':'coupe',
               'suv':'suv',
               'cabrio': 'cabrio',
               'andere': 'other',
               'cabrio': 'convertible',
               # gear box terms
               'manuell': 'manual',
               'automatik': 'automatic',
               # fuel type terms
               'lpg':'lpg',
               'benzin': 'gasoline',
               'diesel':'diesel',
               'cng':'natural gas',
               'hybrid':'hybrid',
               'elektro':'electric',
               'andere':'other',
               # unrepaired damage terms
               'nein':'no',
               'ja':'yes'       
}

Let's clean each of these columns. We will also print out the value counts before and after translating to ensure that the process ran adequately, without compromising our data:

In [11]:
for item in german_columns:
    print('BEFORE TRANSLATING', '\n')
    
    print(autos[item].value_counts(dropna=False))
    
    print('\n', 'TRANSLATING WORDS...')
    print('-'*30)
    
    translate_column(autos, item, translations)
    print(autos[item].value_counts(dropna=False))
    
    print('-'*30)
    print('-'*30)

BEFORE TRANSLATING 

limousine     12859
kleinwagen    10822
kombi          9127
NaN            5095
bus            4093
cabrio         3061
coupe          2537
suv            1986
andere          420
Name: vehicle_type, dtype: int64

 TRANSLATING WORDS...
------------------------------
limousine      12859
small car      10822
combo           9127
NaN             5095
bus             4093
convertible     3061
coupe           2537
suv             1986
other            420
Name: vehicle_type, dtype: int64
------------------------------
------------------------------
BEFORE TRANSLATING 

manuell      36993
automatik    10327
NaN           2680
Name: gear_box, dtype: int64

 TRANSLATING WORDS...
------------------------------
manual       36993
automatic    10327
NaN           2680
Name: gear_box, dtype: int64
------------------------------
------------------------------
BEFORE TRANSLATING 

benzin     30107
diesel     14567
NaN         4482
lpg          691
cng           75
hybrid       

#### Observations
- *The German words have been converted to English, and the value counts remained the same after the translation. We can conclude that the translation was executed correctly without compromising our data.*

### Assigning the Right Data Types

We previously identified some object types that should have been saved as numeric; like the `price` and `odometer` columns. For each of these columns, we will remove the non-numeric characters, convert the column to a numeric data type, then use the `DataFrame.rename()` method to assign descriptive names.

The `price` columns has both the `','` and `'$'` non-numeric characters. The odometer column has `','` and `'km'` as its non-numeric characters.

In [12]:
autos['price'] = (autos['price']
                      .str.replace(',', '')
                      .str.replace('$','')
                      .astype(int)
                 )

autos.rename({'price':'usd_price'}, inplace=True, axis=1)
autos['usd_price'].head(10)

0    5000
1    8500
2    8990
3    4350
4    1350
5    7900
6     300
7    1990
8     250
9     590
Name: usd_price, dtype: int64

In [13]:
autos['odometer'] = (autos['odometer']
                         .str.replace('km', '')
                         .str.replace(',','')
                         .astype(int)
                    )

autos.rename({'odometer':'odometer_km'}, inplace=True, axis=1)
autos['odometer_km'].head(10)

0    150000
1    150000
2     70000
3     70000
4    150000
5    150000
6    150000
7    150000
8    150000
9    150000
Name: odometer_km, dtype: int64

### Exploring the Odometer and Price Columns

Let's continue by exploring the data for outliers. Specifically we are looking for data that doesn't look right, especially in our numeric columns (`odometer_km` and `usd_price`). We will analyze these columns using minimum and maximum values and look for any values that look unrealistically high or low.

In [14]:
autos[['odometer_km', 'usd_price']].describe()

,odometer_km,usd_price
count,50000.000000,5.000000e+04
mean,125732.700000,9.840044e+03
std,40042.211706,4.811044e+05
min,5000.000000,0.000000e+00
25%,125000.000000,1.100000e+03
50%,150000.000000,2.950000e+03
75%,150000.000000,7.200000e+03
max,150000.000000,1.000000e+08


#### Observations

- *The median and 75th percentile value in the `odometer_km` column share the same amount as the max value of 150,000km. The 25th percentile is set at approximately 125,000km. All these indicates that majority (over 75%) of the cars listed are high mileage cars between 125,000 to 150,000km.*

- *The `usd_price` column has a maximum value of 100 million USD. This varies significantly from the median and 75th percentile values of 2,950 and 7,200 USD respectively. Ths standard deviation here is considerably high (about 480,000 USD). We can also notice the minimum price of 0 which can also affect our analysis.* 

We can further explore `odometer_km` by displaying its sorted value counts:

In [15]:
autos['odometer_km'].value_counts().sort_index(ascending=False)

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
40000       819
30000       789
20000       784
10000       264
5000        967
Name: odometer_km, dtype: int64

#### Additional Observations

- *The least mileage observed is 5000km rather than 0km. This is expected because the cars listed on this platform are used cars.*
- *Higher Mileage cars are more common on the listing than lower mileage ones.*
- *Of the 50,000 cars in our dataset, there are over 30,000 (~65%) cars that have odometer readings of 150,000. This could imply that there is an upper mileage limit of 150,000km for cars accepted for listing.*

Next, we will explore `usd_price` and attempt to remove its outliers:

In [16]:
autos['usd_price'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: usd_price, dtype: float64

### Defining Price Outliers

Like we had observed earlier, the `25th`, `50th` and `75th` percentile of the prices are `1,100`, `2,950` and `7,200` USD respectively. We will an outlier detection method that helps us create a “fence” boundary from our 25th and 75th percentiles. Any data beyond these fences will be considered to be outliers

With the information we have gathered, we can calculate our interquartile range `IQR`, which helps us define the upper and lower limits for the list prices to use in our analysis.

```
    Interquartile Range (IQR) = 75th Percentile - 25th Percentile
    Lower Limit = 25th Percentile - 1.5(IQR)
    Upper Limit = 75th Percentile + 1.5(IQR)
    
```
Lets define all these boundaries in the code cell below:

In [17]:
price_25th, price_75th = (1100, 7200)

IQR = price_75th - price_25th
lower_limit = price_25th - (1.5*IQR)
upper_limit = price_75th + (1.5*IQR)

print('Interquartile Range: ', IQR)
print('Lower Limit: ', lower_limit)
print('Upper Limit: ', upper_limit)

Interquartile Range:  6100
Lower Limit:  -8050.0
Upper Limit:  16350.0


Since limits usually extend beyond the data sometimes, it is totally fine if our lower limit is negative. We had noticed earlier that the minimum price value of `0` could also affect our analysis. Hoewer, before we attempt to eliminate the entries with zero prices, we will take a look at how many data points we have with zero prices using the `.value_counts()` method

In [18]:
autos['usd_price'].value_counts().head()

0       1421
500      781
1500     734
2500     643
1000     639
Name: usd_price, dtype: int64

Of the 50,000 rows in our dataset, 1,421 records `(3%)` are listed at zero price. Since this percentage is very low, we can safely remove these entries from our data. To make things easier we will also set our lower_limit to `1` in order to also exclude zero values:

In [19]:
autos = autos[autos['usd_price'].between(1, upper_limit)]

autos['usd_price'].describe()

count    44795.000000
mean      4089.433620
std       3841.429247
min          1.000000
25%       1150.000000
50%       2700.000000
75%       5999.000000
max      16350.000000
Name: usd_price, dtype: float64

#### Observations

- *Our dataset has been reduced to 44,795 rows in total. The 25th, 50th and 75th percentiles are still closer to their initial values before cleaning and we have been able to obtain a better approximation of the maximum price (16,350 USD) that we can use for our analysis.*

Let's now explore the date columns to understand the date range our data covers

### Exploring the Date Columns

There are 5 columns in our dataset that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the information below:

```
- `date_crawled` was added by the crawler
- `last_seen` was added by the crawler
- `ad_created` was obtained from the website
- `registration_month` was obtained from the website
- `registration_year` was obtained from the website

```

Like we observed earlier, `the date_crawled`, `last_seen`, and `ad_created` columns are currently identified as string/object types by pandas. We need to convert them into `datetime` objects so we can understand them quantitatively. `registration_month` and `registration_year` are represented as numeric values, so we can use methods like `Series.describe()` to understand the distribution without any data type conversion.

Let's first understand how the values in the three string columns are formatted. These columns all represent full timestamp values:

In [20]:
autos[['date_crawled', 'ad_created', 'last_seen']].head()

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


#### Observations

- *It appears that the first 10 characters in each column represent the date in the yy-mm-dd format (e.g. 2016-03-12)*

To understand the distribution of the date ranges better, we can extract just the date values using a string method, use `Series.value_counts()` to generate a distribution, then sort by the index (which are the extracted dates in this case).

To select the first 10 characters in each column, we can use `Series.str[:10]`:

### 1. Date Crawled

In [21]:
print(autos['date_crawled'].str[:10].unique().size)

(autos['date_crawled']
     .str[:10]
     .value_counts(normalize=True, dropna=False)
     .sort_index()
)

34


2016-03-05    0.025516
2016-03-06    0.013997
2016-03-07    0.036098
2016-03-08    0.033642
2016-03-09    0.033129
2016-03-10    0.032705
2016-03-11    0.032481
2016-03-12    0.037393
2016-03-13    0.015604
2016-03-14    0.036991
2016-03-15    0.034178
2016-03-16    0.029959
2016-03-17    0.031990
2016-03-18    0.012769
2016-03-19    0.034200
2016-03-20    0.037705
2016-03-21    0.037192
2016-03-22    0.032727
2016-03-23    0.032481
2016-03-24    0.029378
2016-03-25    0.032013
2016-03-26    0.032593
2016-03-27    0.030718
2016-03-28    0.034781
2016-03-29    0.034200
2016-03-30    0.033865
2016-03-31    0.031588
2016-04-01    0.032794
2016-04-02    0.034982
2016-04-03    0.038531
2016-04-04    0.036366
2016-04-05    0.012948
2016-04-06    0.003170
2016-04-07    0.001317
Name: date_crawled, dtype: float64

#### Observations
- *It appears the crawler collected information from the site everyday between March 5, 2016 and April 7, 2016, a period of 34 days. The number of listings crawled per day seems evenly distributed too.*

### 2. Ad Created

In [22]:
print(autos['ad_created'].str[:10].unique().size)

(autos['ad_created']
     .str[:10]
     .value_counts(normalize=True, dropna=False)
     .sort_index()
)

73


2015-08-10    0.000022
2015-09-09    0.000022
2015-11-10    0.000022
2015-12-05    0.000022
2015-12-30    0.000022
                ...   
2016-04-03    0.038799
2016-04-04    0.036701
2016-04-05    0.011653
2016-04-06    0.003237
2016-04-07    0.001183
Name: ad_created, Length: 73, dtype: float64

explanations?

### 3. Last Seen

In [23]:
(autos['last_seen']
     .str[:10]
     .value_counts(normalize=True, dropna=False)
     .sort_index()
)

2016-03-05    0.001161
2016-03-06    0.004576
2016-03-07    0.005693
2016-03-08    0.007903
2016-03-09    0.010068
2016-03-10    0.011162
2016-03-11    0.013037
2016-03-12    0.024936
2016-03-13    0.009376
2016-03-14    0.012859
2016-03-15    0.016274
2016-03-16    0.016966
2016-03-17    0.029043
2016-03-18    0.007568
2016-03-19    0.016475
2016-03-20    0.021230
2016-03-21    0.021185
2016-03-22    0.021922
2016-03-23    0.019109
2016-03-24    0.020449
2016-03-25    0.020002
2016-03-26    0.017234
2016-03-27    0.016207
2016-03-28    0.021699
2016-03-29    0.023016
2016-03-30    0.025472
2016-03-31    0.024333
2016-04-01    0.023306
2016-04-02    0.025226
2016-04-03    0.025673
2016-04-04    0.025047
2016-04-05    0.121286
2016-04-06    0.214332
2016-04-07    0.126175
Name: last_seen, dtype: float64

explanations?

### 4. Registration Year

In [24]:
autos['registration_year'].describe()

count    44795.00000
mean      2003.97339
std         74.81955
min       1000.00000
25%       1999.00000
50%       2003.00000
75%       2007.00000
max       9999.00000
Name: registration_year, dtype: float64

#### Observations

- **_The registration_year column contains some unexpected values:_**

    - *The minimum value is 1000, this is long before 1885 when cars were invented.*
    - *The maximum value is 9999, many years into the future. We are also aware that the listings were last seen by the crawler in 2016. A car can't be first registered after its listing has been seen.* 

Determining the earliest valid year is challenging. However since we know that cars were invented in 1885, we can use the first few decades of 1900s as our lower cut off point. At the higher end, any vehicle with a registration year above 2016 is definitely inaccurate. 

Before we eliminate these rows, let's count the number of listings with cars that fall outside the 1900 - 2016 registration year and see if it's safe to remove these rows entirely, without impacting our analysis too much.

In [25]:
invalid_rows = (~(autos['registration_year'].between(1900, 2016))).sum()
total_rows = autos['registration_year'].shape[0]
percentage_invalid = invalid_rows/total_rows

print('The invalid years constitute {:.1f}% of the total data'.format(percentage_invalid * 100))

The invalid years constitute 4.1% of the total data


Since these years constitute only 4.1% of our total data. We can safely eliminate these records from our analysis:

In [26]:
autos = autos[autos['registration_year'].between(1900, 2016)]

(autos['registration_year']
     .value_counts(normalize=True)
     .sort_values(ascending=False)
     .head(15)
)

2000    0.073140
1999    0.067227
2005    0.067227
2003    0.062129
2004    0.061920
2001    0.060895
2006    0.060127
2002    0.057264
1998    0.054703
2007    0.049512
2008    0.046207
1997    0.045252
2009    0.042087
1996    0.031705
2010    0.028539
Name: registration_year, dtype: float64

#### Observations:
- *It appears that most of the vehicles (a cumulative of 74%) were first registered between the years 1997 to 2009. The remaining the 23% were registered after 2009*

### 5. Registration Month

In [27]:
autos['registration_month'].describe()

count    42959.000000
mean         5.791918
std          3.691957
min          0.000000
25%          3.000000
50%          6.000000
75%          9.000000
max         12.000000
Name: registration_month, dtype: float64

#### Observations:

- *Since the lowest value is set at 0 (presumably denoting January), we would expect the highest value to be at 11 (denoting December). However the highest value we see here is 12. This is quite unexpected.*

We will probe into this further:

In [28]:
(autos['registration_month']
     .value_counts(normalize=True)
     .sort_index()
    
)

0     0.092344
1     0.065760
2     0.060500
3     0.103308
4     0.082846
5     0.083405
6     0.087479
7     0.077679
8     0.064527
9     0.068624
10    0.074932
11    0.068111
12    0.070486
Name: registration_month, dtype: float64

#### Observations:

- **_This looks a bit more challenging:_**
    - *The months are evenly distributed. Hence it would not be a good idea to remove any month in favor of the others*
    - *No month is errously skipped which further complicates what 0 and 12 are supposed to represent.*

Since it is unclear where the error is from, we will drop the `registration_month` column as it could compromise our analysis:

In [29]:
autos = autos.drop('registration_month', axis=1)
autos.shape

(42959, 16)

## Data Analysis

### Exploring Price by Brand

Lets explore the unique values in the brand column, to identify the most common brands in the used car listings:

In [30]:
autos['brand'].value_counts(normalize=True)

volkswagen        0.214763
opel              0.115203
bmw               0.103541
mercedes_benz     0.088131
audi              0.077562
ford              0.073093
renault           0.050769
peugeot           0.032170
fiat              0.027747
seat              0.019088
skoda             0.016993
mazda             0.016085
nissan            0.015782
smart             0.015363
citroen           0.015038
toyota            0.013362
hyundai           0.010545
volvo             0.009334
mitsubishi        0.008706
sonstige_autos    0.008496
honda             0.008334
mini              0.007752
alfa_romeo        0.007007
kia               0.006960
suzuki            0.006378
chevrolet         0.005750
chrysler          0.003748
dacia             0.002863
daihatsu          0.002724
subaru            0.002211
jeep              0.002002
porsche           0.001886
saab              0.001769
daewoo            0.001629
trabant           0.001513
rover             0.001420
land_rover        0.001304
j

#### Observations:

- *The most common brands on the listing is Volkswagen (21%). Opel (about 12%), BMW (10%), Mercedes Benz (9%) and Audi (8%), Ford (7%) and Renaut (5%) follow behind respectively.* 
- *If we summed these percentage, we would find out that these top 7 brands constitute over 72% of the population of listed car brands in our dataset.* 
- *Of the top 7 brands, the top 5 are German (Volkswagen, Opel, BMW, Mercedes and Audi). The constitute about 50% of the entire population of listed brands. The other 2 brands are American (Ford) and French (Renault) brands.*

There other brands only constute a combined 28% of total listings. As a result, we will limit our analysis to brands representing more than 5% of the total. If we take another look at the previous output we will discover that only the top 7 brands meet this criteria.

In [31]:
top_brands = (autos['brand']
                  .value_counts()
                  .head(7)
                  .index
             )

top_brands

Index(['volkswagen', 'opel', 'bmw', 'mercedes_benz', 'audi', 'ford',
       'renault'],
      dtype='object')

In [32]:
aggregate_price = {}

for brand in top_brands:
    
    prices = autos['usd_price'][autos['brand'] == brand]
    mean_price = prices.mean()
    aggregate_price[brand] = int(mean_price)
    
# convert dictionary to a pandas series for easy printing and manipulation
price_series = pd.Series(aggregate_price) 

print(price_series)
print('-'*30)
print('The median of these prices is {:.0f}'.format(price_series.median()))

volkswagen       4183
opel             2712
bmw              5649
mercedes_benz    5259
audi             5703
ford             2944
renault          2281
dtype: int64
------------------------------
The median of these prices is 4183


#### Observations

- *Asides Opel, the German brands (Volkswagen, BMW, Mercedes and Audi) seem more expensive on average than the American or French ones.*
- *The average price for Volkwagen seems to be at the middle point of all these prices, offering an interesting balance between popularity and price.*

### The Top 7 Brands - Exploring Mileage

For the top 7 brands, we will use aggregation to understand the average mileage for those cars and if there's any visible link with mean price. Rather than display both aggregated series objects and visually compare them, we will combine the data from both series objects into a single dataframe (with a shared index) and display the dataframe directly.

We have already converted our `aggregate_price` dictionary into the Pandas series object - `price_series` using the `pd.Series()` constructor. Next we will convert the series object into a Dataframe using the `pd.DataFrame()` constructor:

In [33]:
price_df = pd.DataFrame(data = price_series, columns = ['mean_price_usd'])
price_df

,mean_price_usd
volkswagen,4183
opel,2712
bmw,5649
mercedes_benz,5259
audi,5703
ford,2944
renault,2281


Lets repeat the same process for the `odometer_km` column. We will calculate the average mileage of the cars in the top 7 brands, then construct Series and Dataframe objects from our results.

In [34]:
aggregate_mileage = {}

for brand in top_brands:
    
    mileage = autos['odometer_km'][autos['brand'] == brand]
    mean_mileage = mileage.mean()
    aggregate_mileage[brand] = int(mean_mileage)
    
mileage_series = pd.Series(aggregate_mileage)    
print('The median of these mileages is {:.0f}'.format(mileage_series.median()))
mileage_df = pd.DataFrame(data = mileage_series, columns = ['mean_mileage_km'])

mileage_df

The median of these mileages is 132813


,mean_mileage_km
volkswagen,132813
opel,130500
bmw,138833
mercedes_benz,138411
audi,139701
ford,127004
renault,128892


Finally, we will join the average price and mileage dataframes together to enable us compare their results at a glance. One way we can do this is by assigning `mileage_series` to a new column in the price dataframe:

In [35]:
price_df['mean_mileage_km'] = mileage_series
price_df

,mean_price_usd,mean_mileage_km
volkswagen,4183,132813
opel,2712,130500
bmw,5649,138833
mercedes_benz,5259,138411
audi,5703,139701
ford,2944,127004
renault,2281,128892


#### Observations
- *We would expect the average mileage to fall as the average price of the cars rise. However this is not the case here as we notice the more expensive vehicles having higher mileage, and the cheaper ones recording lower mileages*
- *We see Volkswagen at the median of the mileage values again. It's popularity on the listings might be perhaps attributable to it being the brand that fairly balances price and mileage*

### Popular Brand Model Combinations

We already have information on the most common brands. It would also be interesting to have information on which particular car models contribute to the brand's popularity on the listings.

Panda has a handy `.groupby()` method that enables us to group large amounts of data from a dataframe together, then compute aggregations on these groups. Our action will be a simple two step process:

- First, we will extract the two columns of interest (`brand` and `model`) from our dataframe.

- Next, we will group the distinct occurrences of each brand with each specific model using the `.groupby` method, then compute aggregations of each model's frequency:

In [36]:
# extract the relevant columns from the autos dataframe
brand_model = autos[['brand','model']]

brand_model_grouped = (brand_model
                           .groupby(['brand','model']) # select distinct occurences of each brand and model
                           ['model'].count() # count the distinct models within each brand
                      )

brand_model_grouped

brand       model   
alfa_romeo  145          4
            147         80
            156         88
            159         32
            andere      55
                        ..
volvo       v40         85
            v50         29
            v60          1
            v70         89
            xc_reihe    33
Name: model, Length: 285, dtype: int64

Our results do not look attractive to read. By default, the `.groupby()` method returns a series with the grouped columns as a multi-index. To obtain a more detailed result to read, we will push the grouped columns back into dataframe columns by resetting the indices then assign a column name to our model counts:

In [37]:
# reset multi-index series to dataframe and assign a name to the aggregate column
brand_model_grouped = brand_model_grouped.reset_index(name='count') 
brand_model_grouped

,brand,model,count
0,alfa_romeo,145,4
1,alfa_romeo,147,80
2,alfa_romeo,156,88
3,alfa_romeo,159,32
4,alfa_romeo,andere,55
...,...,...,...
280,volvo,v40,85
281,volvo,v50,29
282,volvo,v60,1
283,volvo,v70,89


This already looks much better, but we only need information on the most popular cars. We need to sort the entire dataframe in descending order based on the values in the `counts` column:

In [38]:
brand_model_grouped.sort_values(by='count', axis=0, ascending=False).head(15)

,brand,model,count
262,volkswagen,golf,3487
22,bmw,3er,2431
268,volkswagen,polo,1604
179,opel,corsa,1589
176,opel,astra,1339
266,volkswagen,passat,1299
13,audi,a4,1126
140,mercedes_benz,c_klasse,1005
23,bmw,5er,991
143,mercedes_benz,e_klasse,813


#### Observations
- **_We'd immediately notice the agreement of this pattern with what we observed in the brands. Volkswagen, Opel, BMW, Mercedes Benz, Audi, Ford, and Renaut remain the top brands:_**
    - *The most popular Volkswagen models are the Golf, Polo and Passat.*
    
    - *The most popular BMW models are the 3er and 5er series.*
    
    - *The most popular Opel models are Corsa and Astra.*
    
    - *Audi is popular with their A-series models (a4, a3 and a6)*
    
    - *Mercedes is popular with the c and e class cars*
    
    - *Ford is popular with its focus and fiesta models while the most popular Renault model is the Twingo*
    
### Investigating Mileage and Price Further

Let's explore how odometer readings influence the average price of a listed car. We will divide our odometer readings into minor intervals then inspect for any interesting price patterns as mileage grows. First we will inspect the `odometer_km` column by sorting its unique values:

In [39]:
sorted(autos['odometer_km'].unique())

[5000,
 10000,
 20000,
 30000,
 40000,
 50000,
 60000,
 70000,
 80000,
 90000,
 100000,
 125000,
 150000]

There are not many unique values in this column, and it appears that the data in this column are round to the nearest thousands (there are no in-betweens). With this in mind, we will split our odometer readings into seven intervals as follows:

```
5000km - 10000km
20000km - 30000km
40000km - 50000km
60000km - 70000km
80000km - 90000km
100000km - 125000km
150000km or more
```
Since these intervals are few and easy to work with, we can manually create a dictionary to hold them:

In [40]:
odometer_groups = {
    5000:'5000km - 10000km',
    10000:'5000km - 10000km',
    20000:'20000km - 30000km',
    30000:'20000km - 30000km',
    40000:'40000km - 50000km',
    50000:'40000km - 50000km',
    60000:'60000km - 70000km',
    70000:'60000km - 70000km',
    80000:'80000km - 90000km',
    90000:'80000km - 90000km',
    100000:'100000km - 125000km',
    125000:'100000km - 125000km',
    150000:'150000km or more'
}

We will extract the relevant columns from our dataset (`odometer_km` and `usd_price`), then assign the resulting dataframe to a variable called *`mileage`*. Next, we will map the values in our odometer_groups dictionary into the odometer column (we can use the handy `translate_column()` function to achieve this).

In [41]:
# extract relevant columns
mileage = autos[['odometer_km', 'usd_price']]

translate_column(mileage,'odometer_km',odometer_groups)
mileage.head(10)

,odometer_km,usd_price
0,150000km or more,5000
1,150000km or more,8500
2,60000km - 70000km,8990
3,60000km - 70000km,4350
4,150000km or more,1350
5,150000km or more,7900
6,150000km or more,300
7,150000km or more,1990
8,150000km or more,250
9,150000km or more,590


Our odometer groups have been successfully mapped to the odometer_km column. We can finally use the `groupby()` method to calculate the average per mileage group:

In [42]:
price_by_mileage =(mileage.groupby('odometer_km')['usd_price'].mean()
                       .reset_index(name='avg_price_usd')
                       .sort_values(by='avg_price_usd', axis=0, ascending=False)
                  )

price_by_mileage['avg_price_usd'] = price_by_mileage['avg_price_usd'].round()
price_by_mileage

,odometer_km,avg_price_usd
3,40000km - 50000km,8109.0
2,20000km - 30000km,7861.0
5,60000km - 70000km,7148.0
6,80000km - 90000km,6224.0
0,100000km - 125000km,5068.0
4,5000km - 10000km,3592.0
1,150000km or more,3298.0


#### Observation

- *Medium mileage cars are the most expensive on average. This comprises vehicles that have travelled between 20,000 - 50,000km (as seen in our table's first and second rows). Their average prices range between 7,861USD to 8,100USD*

- *Moderate to High mileage cars (60,000 - 120,000km) follow behind, with average prices ranging from about 5,000 USD to 7,100 USD*

- *As expected, the highest mileage cars (150,000km or more) are priced the least.* 

- *We observe some unexpected findings with the lowest mileage cars (5000km to 10000km). Though one would expect these cars to be priced the highest, they are priced far lower than expected. It is hard to tell what could have caused this for sure, but this group could comprise cars that got damaged shortly after their purchase.*

### Investigating Price and Unrepaired Damage

We had suggested that the drop in average price of the low mileage cars may be due to some cars that damaged shortly after their purchase. In the next steps, we will find out just how cheap damaged cars are, especially when compared to their non-damaged counterparts. 

Before we proceed, let's take another look at the `unrepaired_damage` column:

In [43]:
print('COUNTS')
print('-'*30)
print(autos['unrepaired_damage'].value_counts(dropna=False))
print("")
print('PERCENTAGES')
print('-'*30)
print(autos['unrepaired_damage'].value_counts(dropna=False, normalize=True))

COUNTS
------------------------------
no     30361
NaN     8112
yes     4486
Name: unrepaired_damage, dtype: int64

PERCENTAGES
------------------------------
no     0.706744
NaN    0.188831
yes    0.104425
Name: unrepaired_damage, dtype: float64


#### Observation

- *A huge percentage of cars (about 70%) in our dataset are in good condition. 10% of the car listings have unrepaired damages and we lack enough information on the status of the remaining 19%. The null values in this group will not be useful for our analysis.*

We can create our working dataframe from the `usd_price` and `unrepaired_damage` columns, then drop the rows with null values:

In [44]:
price_damage_table = autos[['usd_price','unrepaired_damage']]
print(price_damage_table.shape)
price_damage_table.dropna(axis=0, inplace=True)
print(price_damage_table.shape)
print(price_damage_table.shape[0] + 8112) # verify the number of dropped columns with the no of NaN values (8112)
price_damage_table

(42959, 2)
(34847, 2)
42959


,usd_price,unrepaired_damage
0,5000,no
1,8500,no
2,8990,no
3,4350,no
4,1350,no
...,...,...
49992,4800,no
49994,5000,no
49996,1980,no
49997,13200,no


Next we will group by `unrepaired_damage` while computing the average USD prices for both the 'yes' and 'no' categories:

In [45]:
price_to_damage = (price_damage_table.groupby('unrepaired_damage')['usd_price'].mean()
                      .reset_index(name='avg_price_usd')
                  )
price_to_damage['avg_price_usd'] = price_to_damage['avg_price_usd'].round()
price_to_damage

,unrepaired_damage,avg_price_usd
0,no,4851.0
1,yes,1972.0


#### Observation

- *There is a huge variation in price between this two groups. We can safely say that, on average, cars in good condition are **2.5 times** more expensive than those with unrepaired damages.*

Now that we have established that unrepaired_damage has a significant effect on price, we can go further to explore the reason for the **shocking low prices observed for the lowest mileage cars (5,000 - 10,000km)**:

### Were the Lowest Mileage Cars Affected By Some Sort of Early Damage?

To conduct this investigation, we need two columns from our cleaned Dataframe (`odometer_km` and `unrepaired_damage`). We will perform our analysis in four steps:

1. We will collect the two columns into a seperate dataframe.

2. We will use the `translate_column` function to map odometer_km to the respective mileage intervals.

3. We will remove the null values from unrepaired damage and map the column to dummy variables: 1 for 'yes', 0 for 'no'.

4. Finally, we will use the `.groupby()` method to compute the damaged cars ratio. This ratio represents the weight of unrepaired_damage to the number of cars in each mileage interval (we can do this by calculating the mean unrepaired damage).

Lets take this steps one after the other:

#### 1. Collect the required columns into a seperate dataframe

In [46]:
working_df = autos[['odometer_km', 'unrepaired_damage']]

#### 2. Map the Odometer column to the respective mileage intervals

In [47]:
translate_column(working_df,'odometer_km',odometer_groups)

#### 3. Remove the null values from unrepaired damage and map the column to dummy variables 1 and 0

In [48]:
working_df.dropna(axis=0, inplace=True)

translate_column(working_df,'unrepaired_damage',{'yes':1,'no':0})

#### 4. Compute the average value of unrepaired_damage to number of cars in each mileage interval

In [49]:
grouped_df = (working_df.groupby('odometer_km')['unrepaired_damage'].mean()
                 .reset_index(name='dmg_cars_ratio')
             )

grouped_df['dmg_cars_ratio'] = grouped_df['dmg_cars_ratio'].round(3)
grouped_df.sort_values(by='dmg_cars_ratio', ascending=False, axis=0, inplace=True)
grouped_df

,odometer_km,dmg_cars_ratio
4,5000km - 10000km,0.212
1,150000km or more,0.150
0,100000km - 125000km,0.095
6,80000km - 90000km,0.081
5,60000km - 70000km,0.068
2,20000km - 30000km,0.062
3,40000km - 50000km,0.053


#### Observations
- *It is evident that the prices for the low mileage cars were affected by the significant cases of unrepaired damage within the group. This further confirms our suspicions that some of the low mileage vehicles were new vehicles that may have been damaged within a short duration of purchase*

There are some additional insights in the table above. However, before making further observations, we will merge our `avg_price_usd` from the previous `price_by_mileage` dataframe into our new dataframe to obtain a more robust picture:

In [52]:
grouped_df['avg_price_usd'] = price_by_mileage['avg_price_usd']
grouped_df

,odometer_km,dmg_cars_ratio,avg_price_usd
4,5000km - 10000km,0.212,3592.0
1,150000km or more,0.150,3298.0
0,100000km - 125000km,0.095,5068.0
6,80000km - 90000km,0.081,6224.0
5,60000km - 70000km,0.068,7148.0
2,20000km - 30000km,0.062,7861.0
3,40000km - 50000km,0.053,8109.0


#### Observations

- *The price trend we noticed before can be better explained when considering unrepaired damage. Here we see the best-priced cars having the lowest weight of unrepaired damage, while the poorly priced vehicles have a higher occurrence of damages within their group.*

- *The degree of usefulness of unrepaired damage when compared to odometer readings for price evaluation is contrary to what we initially thought. We had expected newer vehicles to have lesser mileages and better prices overall.*

- *However, it turns out that prices will rise or drop, as the weight of unrepaired damage reduces or increases respectively (inverse correlation). This could even occur irrespective of how impressive a car's mileage is.*